# Probability-Based Interaction Objective

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.ioff()
from IPython.display import HTML

In [3]:
import mantrap
import numpy as np
import torch

env = mantrap.environment.Trajectron(
    ego_type=mantrap.agents.IntegratorDTAgent, 
    ego_position=torch.zeros(2), 
    ego_velocity=torch.zeros(2),
    dt=0.4
)
env.add_ado(position=torch.tensor([-5, 2.0]), velocity=torch.tensor([1, 0]))

# Compute ego trajectories to base comparison on.
t_horizon = 9
ego_controls = [
    torch.zeros((t_horizon, 2), requires_grad=True),
    torch.tensor([[0.0, -1.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
    torch.tensor([[1.0, -1.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
    torch.tensor([[-1.0, 0.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
    torch.tensor([[0.0, 1.0] * t_horizon], requires_grad=True).view(t_horizon, 2),
]
colors = ["red", "blue", "green", "yellow", "orange"]
assert len(ego_controls) == len(colors)

n = len(ego_controls)
ego_trajectories = [env.ego.unroll_trajectory(controls, dt=env.dt) for controls in ego_controls]

# Compute trajectory predictions in response to different ego actions.
ado_trajectory_wo = env.predict_wo_ego(t_horizon=t_horizon + 1).squeeze()
ado_trajectories_w = [env.predict_w_trajectory(trajectory).squeeze() for trajectory in ego_trajectories]

In [5]:
from mantrap.visualization import visualize_trajectories

labels_all = ["ado_wo", *[f"ado_{i + 1}" for i in range(n)], *[f"ego_{i + 1}" for i in range(n)]]
colors_all = ["black", *colors, *colors]

HTML(visualize_trajectories([ado_trajectory_wo, *ado_trajectories_w, *ego_trajectories],
                             labels=labels_all, colors=colors_all, env_axes=env.axes))

In [4]:
cost = mantrap.modules.InteractionProbabilityModule(env=env, t_horizon=t_horizon, weight=1.0)

print("Objectives")
for trajectory, color in zip(ego_trajectories, colors):
    objective = cost.objective(trajectory, ado_ids=env.ado_ids, tag="test")
    print(f"{color} ==> {objective}")
    
print("\nGradients (L2-norm)")
for controls, trajectory, color in zip(ego_controls, ego_trajectories, colors):
    gradient = np.linalg.norm(cost.gradient(trajectory, grad_wrt=controls, ado_ids=env.ado_ids, tag="test"))
    print(f"{color} ==> {gradient}")

Objectives
red ==> 12.584221839904785
blue ==> 20.56964111328125
green ==> 10.537171363830566
yellow ==> 22.39353370666504
orange ==> 23.250043869018555

Gradients (L2-norm)
red ==> 7.1656999588012695
blue ==> 9.328543663024902
green ==> 9.09726333618164
yellow ==> 7.572229385375977
orange ==> 4.465150356292725
